<a href="https://colab.research.google.com/github/doanquanvietnamca/pytorch-deploy/blob/master/pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torch import torch
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [0]:
from torch import optim,nn
from torch.utils.data import DataLoader
from torchvision import transforms

In [0]:
from torchvision import datasets

In [0]:
mnist_train = datasets.FashionMNIST(root='Fashion-MNIST/processed/training.pt', train=True, transform=transforms.ToTensor(), download=True)

0it [00:00, ?it/s]

26427392it [00:04, 5894245.78it/s]                               


Extracting Fashion-MNIST/processed/training.pt/FashionMNIST/raw/train-images-idx3-ubyte.gz to Fashion-MNIST/processed/training.pt/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 38003.14it/s]                           
0it [00:00, ?it/s]

Extracting Fashion-MNIST/processed/training.pt/FashionMNIST/raw/train-labels-idx1-ubyte.gz to Fashion-MNIST/processed/training.pt/FashionMNIST/raw


4423680it [00:02, 1702858.60it/s]                            
0it [00:00, ?it/s]

Extracting Fashion-MNIST/processed/training.pt/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to Fashion-MNIST/processed/training.pt/FashionMNIST/raw


8192it [00:00, 15398.18it/s]            

Extracting Fashion-MNIST/processed/training.pt/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to Fashion-MNIST/processed/training.pt/FashionMNIST/raw
Processing...
Done!


In [0]:
mnist_val = datasets.FashionMNIST(root='Fashion-MNIST/processed/training.pt', train=False, transform=transforms.ToTensor(), download=True)

In [0]:
train_loader = DataLoader(mnist_train, batch_size=32)
val_loader = DataLoader(mnist_val, batch_size=32)


In [0]:
import torch.nn.functional as F
import numpy as np
class learner(nn.Module):
  def __init__(self, input, output):
    super(learner, self).__init__()
    self.liner1 = nn.Linear(input,128)
    self.liner2 = nn.Linear(128,64)
    self.liner3 = nn.Linear(64,output)
  def forward(self,x):
    y1 = F.relu(self.liner1(x))
    y2 = F.relu(self.liner2(y1))
    y3 = self.liner3(y2)
    return F.softmax(y3,dim=1)


In [0]:

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return torch.from_numpy(np.eye(num_classes, dtype='uint8')[y]).float()
    

In [0]:
to_categorical(2,10)

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
model = learner(784,10)

loss_F = nn.CrossEntropyLoss()
optimer = optim.SGD(params=model.parameters(), lr=0.001)

In [0]:
MLOSS = []
from tqdm import tqdm

def train(epochs):
  for i in range(epochs):
    for x,y in tqdm(iter(train_loader)):
      optimer.zero_grad()

      ypred= model(x.view(32,784))
      #print(ypred.shape, y.shape)
      loss = loss_F( ypred,y)
      
      loss.backward()
      optimer.step()
    MLOSS.append(loss_F(ypred, y))
  return MLOSS
train(60)

 28%|██▊       | 532/1875 [00:02<00:05, 227.56it/s]

In [0]:
def val():
  correct = 0
  predict= []
  all = 0
  for x,y in tqdm(iter(val_loader)):
    ypred = model(x.view(x.shape[0],784))
    ypred = np.argmax(ypred.detach().numpy(), axis=1)
    correct += (ypred == y.detach().numpy()).sum()
    all += x.shape[0]
    predict.append(ypred)
  print("\n", correct/all)
  return predict
predict = val()

100%|██████████| 313/313 [00:00<00:00, 322.54it/s]


 0.761


In [0]:
val_loader.y

AttributeError: ignored

In [0]:
train_loader.y 

AttributeError: ignored